[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain 学院

欢迎来到 LangChain 学院！

## 背景

在 LangChain，我们的目标是让构建 LLM 应用变得简单。其中一种可以构建的 LLM 应用是代理（agent）。代理的开发受到广泛关注，因为它们可以自动执行一系列此前无法完成的任务。

但在实际中，构建能够可靠执行这些任务的系统非常困难。在我们与用户将代理投入生产环境的过程中，我们发现通常需要更多的控制。例如，你可能需要代理始终先调用某个特定工具，或根据状态使用不同的提示（prompt）。

为了解决这个问题，我们构建了 [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview) —— 一个用于构建代理和多代理应用的框架。LangGraph 与 LangChain 包是分离的，其核心设计理念是帮助开发者在代理工作流中引入更精确的控制，以适应真实世界系统的复杂性。

## 课程结构

本课程由若干模块组成，每个模块聚焦 LangGraph 的某个主题。你会看到每个模块对应的文件夹，其中包含一系列笔记本（notebook）。每个笔记本通常配有讲解视频以辅助理解，但笔记本本身也能独立阅读，包含了必要的说明。每个模块文件夹还包含一个 `studio` 文件夹，里面有可以加载到 [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio)（我们用于构建 LangGraph 应用的 IDE）中的图（graphs）。

## 环境准备

在开始之前，请先按照 `README` 中的说明创建虚拟环境并安装依赖。

## 聊天模型

在本课程中，我们将使用聊天模型（Chat Models），它们以一系列消息作为输入并返回消息作为输出。LangChain 通过[第三方集成](https://docs.langchain.com/oss/python/integrations/chat)支持多种模型。默认情况下，课程将使用 [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai)，因为它既流行又性能良好。请确保已设置 `OPENAI_API_KEY` 环境变量。

下面我们将检查 `OPENAI_API_KEY` 是否设置，如未设置会提示你输入。

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 设置第三方 API Key
_set_env("OPENAI_API_KEY")

# 设置第三方 API Base URL（必需）
# 例如: 
# DeepSeek: https://api.deepseek.com/v1
# 智谱 AI: https://open.bigmodel.cn/api/paas/v4
# 月之暗面 (Kimi): https://api.moonshot.cn/v1
# 阿里云百炼: https://dashscope.aliyuncs.com/compatible-mode/v1
_set_env("OPENAI_API_BASE")

[这里](https://docs.langchain.com/oss/python/langchain/models) 有一份关于聊天模型可用功能的实用指南，下面我们会展示一些要点。如果你已经按照 README 中的说明运行了 `pip install -r requirements.txt`，那么你已经安装了 `langchain-openai` 包。使用该包我们可以实例化 `ChatOpenAI` 模型对象。你可以在 [这里](https://openai.com/api/pricing/) 查看各种模型的定价。笔记本中默认会使用 `gpt-4o`，因为它在质量、价格和速度之间有良好平衡，但你也可以选择价格更低的 `gpt-3.5` 系列或其他更新的模型。

有[一些常见参数](https://docs.langchain.com/oss/python/langchain/models#parameters) 可以用于聊天模型。其中两个最常用的是：

* `model`：模型名称
* `temperature`：采样温度

`temperature` 控制模型输出的随机性或创造性，温度低（接近 0）时输出更确定、更聚焦，适用于需要准确或基于事实的任务；温度高（接近 1）时更适合创意类任务或生成多样化回复。

In [2]:
from langchain_openai import ChatOpenAI

# 使用环境变量中的配置
# OPENAI_API_KEY 和 OPENAI_API_BASE 会自动从环境变量中读取
dsv3_2_chat = ChatOpenAI(model="deepseek-ai/DeepSeek-V3.2-Exp", temperature=0)
MiniMax_M2_chat = ChatOpenAI(model="MiniMaxAI/MiniMax-M2", temperature=0)

# 注意：请根据你使用的第三方服务修改 model 参数
# 例如：
# DeepSeek: model="deepseek-chat"
# 智谱 AI: model="glm-4"
# 月之暗面: model="moonshot-v1-8k"
# 阿里通义千问: model="qwen-turbo"

LangChain 中的聊天模型有许多[默认方法](https://reference.langchain.com/python/langchain_core/runnables)。在大多数情况下，我们会使用：

* [stream](https://docs.langchain.com/oss/python/langchain/models#stream)：流式返回响应片段
* [invoke](https://docs.langchain.com/oss/python/langchain/models#invoke)：对输入调用模型

如前所述，聊天模型以[消息（messages）](https://docs.langchain.com/oss/python/langchain/messages)作为输入。消息有一个角色（描述谁在说话）和内容属性（content）。我们之后会更详细讨论，这里先展示基本用法。

In [3]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
dsv3_2_chat.invoke(messages)

AIMessage(content="Hello world! 👋 \n\nIt's great to see you! How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 8, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'id': '019a9c87aabab3722475eb9fd4fc8b77', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ccfb2fc3-9d63-4424-a159-8400f99abd8f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 20, 'total_tokens': 28, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}})

我们会得到一个 `AIMessage` 响应。另请注意，我们也可以直接用字符串调用聊天模型。当以字符串作为输入时，该字符串会被转换为一个 `HumanMessage`，然后传递给底层模型。

In [ ]:
dsv3_2_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CSWGCXlVYTEWoHAFm3GXIIgUO1gCb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--43b070a6-7676-4aa1-984c-c0cd43d45c1e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
MiniMax_M2_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CSWGDY5KePqihRcWDX3IGEZfSoyld', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f7963c03-f8b2-4eba-bc7f-4d58520fa661-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

不同聊天模型之间的接口是一致的，模型通常在每个笔记本启动时初始化一次。

因此，如果你更偏好其他提供商，可以在不更改下游代码的情况下轻松切换模型。

## 搜索工具

在 README 中你还会看到 [Tavily](https://tavily.com/)，它是一个为 LLM 和 RAG 优化的搜索引擎，旨在提供高效、快速且持久的搜索结果。如前所述，注册很容易且提供了宽松的免费额度。第 4 模块中的某些课程将默认使用 Tavily，但当然如果你愿意，也可以在代码中替换为其他搜索工具。

In [4]:
_set_env("TAVILY_API_KEY")

In [ ]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is LangGraph?"})
type(data)
search_docs = data.get("results", data)

In [5]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
  'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.',
  'score': 0.9999573,
  'raw_content': None},
 {'url': 'https://www.geeksforgeeks.org/machine-learning